## Kate Groschner
## AY250 Homework 3

# 1) Monty: The Python Siri

Let's make a Siri-like program (call it Monty!) with the following properties:
   - record your voice command
   - use a webservice to parse that sound file into text
   - based on what the text, take three different types of actions:
       - send an email to yourself
       - do some math
       - tell a joke

So for example, if you say "Monty: email me with subject hello and body goodbye", it will email you with the appropriate subject and body. If you say "Monty: tell me a joke" then it will go to the web and find a joke and print it for you. If you say, "Monty: calculate two times three" it should response with printing the number 6.

Hint: you can use speed-to-text apps like Houndify (or, e.g., Google Speech https://cloud.google.com/speech/) to return the text (but not do the actions). You'll need to sign up for a free API and then follow documentation instructions for using the service within Python. 

# Question 1
My strategy for this question is to use the audio recording script Prof. Bloom showed in class. Send this to a houndify to parse, this portion was written mostly using the documentation provided by houndify. Taking the output text from houndify is put into a list format after determining the type of request. This part of the script is essentially a decision tree that leads to the last step that either a) uses stmp to send an email or b) does the math, or finally c) uses beautifulsoup to go to a website and get jokes

The math portion can take in directions in several ways:
    addition can either be directed as "number plus number" or "add number and number"
    subtraction can be "number minus number" or "subtract number from number"
    multiplication can be "number times number" or "multiply number and number"
    division is given as "number divided by number"

The email segment requires the following verbal format:
    "email me subject FILL IN and body FILL IN"
    As long as you say email before any other command including joke, add, times etc. the program will email you and 
    not follow the later command
    
The joke segment only requires that you say "joke" and do not activate any of the other functionalities 

    

In [202]:
#Audio recording portion taken from lecture notes and cross referenced with the PyAudio example documentation
import pyaudio 
import wave

def ask_monty():
    """This function records 16 bit audio that is saved as a file called ask_monty.wav which is can then be fed
    into the audio2text file"""
    chunk = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 16000
    RECORD_SECONDS = 5
    WAVE_OUTPUT_FILENAME = "ask_monty.wav"
    p = pyaudio.PyAudio()
    stream = p.open(format = FORMAT,
        channels = CHANNELS,
        rate = RATE,
        input = True,
        frames_per_buffer = chunk)
    print("* recording")
    frames = []
    for i in range(0, int(RATE / chunk * RECORD_SECONDS)):
        data = stream.read(chunk)
        frames.append(data)
    print("* done recording")
    stream.close()
    p.terminate()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

In [208]:
#define a function to do the speech to text inquiry
import houndify
import wave
import time
import my_credentials as mc

class MyListener(houndify.HoundListener):
  def onFinalResponse(self, response):
    print("Final response: " + str(response))
  def onError(self, err):
    print("Error: " + str(err))


def audio2text():
    """ This function sends the ask_monty.wav file generated by ask_monty() to houndify where is parsed and returns
    JSON data. It requires that you have a file labelled my_credentials.py with houndify credentials labelled CLIENT_ID
    and CLIENT_KEY"""
    AUDIO_FILE = "ask_monty.wav"
    BUFFER_SIZE = 512
    client = houndify.StreamingHoundClient(mc.CLIENT_ID, mc.CLIENT_KEY, "test_user")
    audio = wave.open(AUDIO_FILE)
    if audio.getsampwidth() != 2:
      print("%s: wrong sample width (must be 16-bit)" % AUDIO_FILE)
    if audio.getframerate() != 8000 and audio.getframerate() != 16000:
      print("%s: unsupported sampling frequency (must be either 8 or 16 khz)" % AUDIO_FILE)
    if audio.getnchannels() != 1:
      print("%s: must be single channel (mono)" % AUDIO_FILE)

    client.setSampleRate(audio.getframerate())
    client.start(MyListener())

    while True:
      samples = audio.readframes(BUFFER_SIZE)
      if len(samples) == 0: break
      if client.fill(samples): break


    result = client.finish() # returns either final response or error
    return result

In [209]:
#defining a function to classify the output of houndify
def sntc_type(result):
    """This function takes in the result returned from houndify and will return a list of the words in that text \
    and a classification of whether asking for math, email, or joke"""
    sentence = result['AllResults'][0]['FormattedTranscription']
    if 'email' in sentence:
        sntcType = 'email'
    elif 'plus' in sentence:
        sntcType = "addition"
    elif 'add' in sentence:
        sntcType = "addition"
    elif 'time' in sentence:
        sntcType = 'multiplication'
    elif 'multiply' in sentence:
        sntcType = 'multiplication'
    elif 'minus' in sentence:
        sntcType = 'subtraction'
    elif 'subtract' in sentence:
        sntcType = 'subtraction'
    elif 'divid' in sentence:
        sntcType = 'division'
    elif 'joke' in sentence:
        sntcType = 'joke'
    else:
        print('Inquiry not of valid type. Please record your question again.')
        sntcType = 'Invalid'
    sntcList = sentence.split(" ")
    return sntcType, sntcList

In [216]:
#defining functions to send email.
import smtplib
import os
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders
from email.utils import COMMASPACE, formatdate
import my_credentials as mc

def mail(subject, text, files=[]):
    msg = MIMEMultipart()
    msg["From"] = mc.GMAIL_USERNAME
    msg["To"] = COMMASPACE.join(mc.GMAIL_USERNAME)
    msg["Date"] = formatdate(localtime=True)
    msg["Subject"] = subject
    msg.attach(MIMEText(text))
    for file in files:
        part = MIMEBase("application", "octet-stream")
        part.set_payload( open(file,"rb").read() )
        Encoders.encode_base64(part)
        part.add_header("Content-Disposition", "attachment; filename='%s'"
                       % os.path.basename(file))
        msg.attach(part)
    # Note that we need to designate the remote SMTP server we want to use.
    mailServer = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    mailServer.login(mc.GMAIL_USERNAME, mc.GMAIL_PASSWORD)
    mailServer.starttls()
    mailServer.sendmail(mc.GMAIL_USERNAME, mc.GMAIL_USERNAME, msg.as_string())
    mailServer.close()

def monty_mail(sntcList):
    idx_sub = sntcList.index('subject')
    idx_bod = sntcList.index('body')
    subject = ' '.join(sntcList[idx_sub+1:idx_bod])
    text = ' '.join(sntcList[idx_bod+1:])
    mail(subject,text)
    

In [217]:
tst = ['Monty','email', 'me', 'subject','dad', 'jokes','body','dad','jokes','are','best','jokes']
monty_mail(tst)

AttributeError: module 'my_credentials' has no attribute 'GMAIL_USERNAME'

In [ ]:
#Here I define a math function to call to handle requests to do math
def monty_add(listIn):
    if 'plus' in listIn:
        idx = listIn.index('plus')
        answer = listIn[idx-1] + listIn[idx+1]
        print('The answer is {}'.format(answer))
    elif 'add' in listIn:
        idx = listIn.index('add')
        answer = listIn[idx+1] + listIn[idx+3]

In [195]:
#Portion of the code to grab jokes from a website.
def monty_joke():
    from urllib.request import urlopen
    from bs4 import BeautifulSoup
    import json
    from random import randint
    jk_url = 'http://pun.me/pages/dad-jokes.php'
    response = urlopen(jk_url)
    jk = response.read().decode('-utf-8')
    response.close()
    jk_soup = BeautifulSoup(jk,"html.parser")
    items = jk_soup.findAll("li")
    #now that I have a list of all the content items from the page I want to strip the html off
    #and the items that are buttons to other pages and just have strings
    jokes  = [i.text.strip() for i in items if "<a href=" not in str(i)]
    idx = randint(0,39)
    print(jokes[idx])

# 2) Write a program that identifies musical notes from sound (AIFF) files. 

  - Run it on the supplied sound files (12) and report your program’s results. 
  - Use the labeled sounds (4) to make sure it works correctly. The provided sound files contain 1-3 simultaneous notes from different organs.
  - Save copies of any example plots to illustrate how your program works.
  
  https://piazza.com/berkeley/spring2018/ay250class13410/resources -> Homeworks -> hw3_sound_files.zip

Hints: You’ll want to decompose the sound into a frequency power spectrum. Use a Fast Fourier Transform. Be care about “unpacking” the string hexcode into python data structures. The sound files use 32 bit data. Play around with what happens when you convert the string data to other integer sizes, or signed vs unsigned integers. Also, beware of harmonics.